In [112]:
#python libraries
import os
import re
from datetime import datetime
import pytz

#other ML libraries
import pandas as pd

#ploting
import matplotlib.pyplot as plt
%matplotlib inline

In [113]:
result_file = open("./cfg/house.csv")

#Skipping first two lines
result_file.readline()
result_file.readline()

period = result_file.readline().split()[2:10]

zone_names = result_file.readline().split()
temp_map = {key:[] for key in zone_names}
start_day, start_month, year = period[3].split('/')
start_day, start_month, year = int(start_day), int(start_month), int(year) + 2000
counter = 1
while measured_temp := result_file.readline().split():
    for i, value in enumerate(measured_temp):
        if(i == 0):
            time = value
            hour, min = time.split('h')
            hour, min = int(hour), int(min)
            day = start_day + counter//24
            if day > 31: #TODO: Brute force, FIX
                day = 1
            month = start_month + counter//(24*31)
            datetime_obj = datetime(year=year, month=month, day=day, hour=hour, minute=min, tzinfo=pytz.timezone('Etc/GMT-2'))
            temp_map[zone_names[i]].append(datetime_obj)
        else:
            temp = value
            temp_map[zone_names[i]].append(float(temp))
    counter += 1

In [114]:
#Import Disruptive & Netatmo Sensor Data

# assign variables
directory = './storedData/'
sensor_data = pd.DataFrame()
sensor_data['timestamp'] = None #in order to merge based on it 

measurement_types_skip = ["Door", "Water", "Humidity_temperature", "Humidity_humidity"]


# iterate over files in that directory and concat into 1 big dataframe based on timestamps
for filename in os.listdir(directory): 
    file = os.path.join(directory, filename)
    if os.path.isfile(file):    
        df = pd.read_csv(file)
        df.drop(columns=df.columns[0], inplace=True) # dropping unnamed coloumn for indexing in .cvs file

        df['timestamp'] = pd.to_datetime(df['timestamp'],  utc=True).dt.tz_convert('Etc/GMT-2') #set timezone correctly if not correct in local time
        df.set_index("timestamp", inplace=True)
        for measurement_name in df.columns:
            measurement_frame = df[measurement_name].to_frame()

            measurement_name_list = re.findall("([A-Z0-9][^A-Z]*)", measurement_name) # Splitting string on capital letters
            measurement_type = measurement_name_list[-1]
            if measurement_type in measurement_types_skip: #Temperature measurements does not have temperature in their name
                continue

            measurement_type = "Temperature"
            measurement_name_list.append(measurement_type)

            #Renaming coloumn for consistent naming
            coloumn_name = f"{''.join(measurement_name_list[:-1])}_{measurement_type.lower()}"
            measurement_frame.rename(columns={measurement_frame.columns[0]: coloumn_name}, inplace=True)

            measurement_frame  = measurement_frame.resample('1H').mean().interpolate() #Resample to managable timestamps by taking mean of 5 minute slots and linear interpolating NaNs, i.e. 5, 10, 15, 20 minutes etc..
            sensor_data = pd.merge(sensor_data, measurement_frame, on='timestamp', how="outer")  
    

sensor_data.set_index("timestamp", inplace=True)
sensor_data.sort_index(axis=0, inplace=True) #Sorting by timestamp
sensor_data.sort_index(axis=1, inplace=True) #Sorting columns alpabetically

# # #linearly interpolate temperature values:
sensor_data = sensor_data.loc[:, sensor_data.columns.str.endswith('temperature')].interpolate().bfill().ffill()

sensor_data["0BTLivingRoom_temperature"] = (sensor_data["0BTCeiling_temperature"] + sensor_data["0BTFloor_temperature"])/2
sensor_data = sensor_data.drop("0BTFloor_temperature", axis=1).drop("0BTCeiling_temperature", axis=1).sort_index(axis=1)

sensor_data["1Entrance_temperature"] = (sensor_data["1Entrance1_temperature"] + sensor_data["1StairBottom_temperature"])/2
sensor_data = sensor_data.drop("1Entrance1_temperature", axis=1).drop("1StairBottom_temperature", axis=1).sort_index(axis=1)

In [115]:
import pandas as pd
sim_temp = pd.DataFrame(temp_map)
sim_temp["Time"] = pd.to_datetime(sim_temp['Time'],  utc=True).dt.tz_convert('Etc/GMT-2')
sim_temp = sim_temp.set_index('Time')

rename_map = {"Time": "timestamp",
              "Ambientdb(C)(C)": "1OutdoorEntrance_temperature",
              "gfSleepdbT(C)": "0BRoom_temperature",
              "gfLivRoomdbT(C)": "0BTLivingRoom_temperature",
              "1fEntrancedbT(C)": "1Entrance_temperature",
              "1fSleep1dbT(C)": "1GuestRoom_temperature",
              "1fSleep2dbT(C)": "1MainRoom_temperature",
              "2fLivRoomdbT(C)": "2LivingRoomCenter_temperature",
              "2fOfficedbT(C)": "2OfficeDesk_temperature"}

sim_temp = sim_temp.reset_index()[rename_map.keys()].rename(columns=rename_map).set_index("timestamp").sort_index(axis=1)
sim_temp.head(2)

0BRoom_temperature  0BTLivingRoom_temperature  \
timestamp                                                                  
2022-07-01 01:00:00+02:00               29.29                      28.29   
2022-07-01 02:00:00+02:00               28.64                      27.94   

                           1Entrance_temperature  1GuestRoom_temperature  \
timestamp                                                                  
2022-07-01 01:00:00+02:00                  32.33                   32.64   
2022-07-01 02:00:00+02:00                  31.93                   31.69   

                           1MainRoom_temperature  \
timestamp                                          
2022-07-01 01:00:00+02:00                  32.26   
2022-07-01 02:00:00+02:00                  31.95   

                           1OutdoorEntrance_temperature  \
timestamp                                                 
2022-07-01 01:00:00+02:00                         20.85   
2022-07-01 02:00:00+02:00                         20.45   

                           2LivingRoomCenter_temperature  \
timestamp                                                  
2022-07-01 01:00:00+02:00                          36.70   
2022-07-01 02:00:00+02:00                          35.31   

                           2OfficeDesk_temperature  
timestamp                                           
2022-07-01 01:00:00+02:00                    36.88  
2022-07-01 02:00:00+02:00                    35.80

In [116]:
sensor_data = sensor_data.loc[(sensor_data.index >= sim_temp.index[0]) & (sensor_data.index <= sim_temp.index[-1])]
sensor_data = sensor_data.reset_index()[rename_map.values()].set_index("timestamp").sort_index(axis=1)
sensor_data.head(2)

0BRoom_temperature  0BTLivingRoom_temperature  \
timestamp                                                                  
2022-07-01 01:00:00+02:00           24.533333                  22.387500   
2022-07-01 02:00:00+02:00           24.212500                  22.335455   

                           1Entrance_temperature  1GuestRoom_temperature  \
timestamp                                                                  
2022-07-01 01:00:00+02:00              24.769886                 25.0250   
2022-07-01 02:00:00+02:00              24.774432                 25.1125   

                           1MainRoom_temperature  \
timestamp                                          
2022-07-01 01:00:00+02:00                25.3875   
2022-07-01 02:00:00+02:00                25.4250   

                           1OutdoorEntrance_temperature  \
timestamp                                                 
2022-07-01 01:00:00+02:00                        20.625   
2022-07-01 02:00:00+02:00                        20.300   

                           2LivingRoomCenter_temperature  \
timestamp                                                  
2022-07-01 01:00:00+02:00                      27.059091   
2022-07-01 02:00:00+02:00                      26.970000   

                           2OfficeDesk_temperature  
timestamp                                           
2022-07-01 01:00:00+02:00                27.068182  
2022-07-01 02:00:00+02:00                26.866667

In [120]:
print("MSE:")
pow((sensor_data - sim_temp), 2).sum()/len(sensor_data)

MSE:


0BRoom_temperature               45.703195
0BTLivingRoom_temperature        40.329041
1Entrance_temperature            28.402615
1GuestRoom_temperature           53.900449
1MainRoom_temperature            39.748448
1OutdoorEntrance_temperature      0.189247
2LivingRoomCenter_temperature    67.407811
2OfficeDesk_temperature          69.595831
dtype: float64

In [119]:
axs = sensor_data.plot(subplots=True, layout=(4, 2), figsize=(20, 30), title=list(sensor_data.columns), ylabel="°C")
sim_temp.plot(subplots=True, ax=axs, linestyle="dashed", legend="T")

for ax in axs:
    ax[0].legend(["Measured", "Predicted"])
    ax[1].legend(["Measured", "Predicted"])